# GEOPHYS 257 (Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

## Multicore Parallelization of Matrix Multiplication Using Numba

In this lab we will be using Numba to accelerate matrix-matrix multiplications by exploiting parallelism. Even most laptops today have Multicore CPUs, where a *core* is a microprocessor, and each core is usually a copy of the each other core. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba](https://numba.readthedocs.io/en/stable/index.html): Documentation
* [Numba in 30 min](https://youtu.be/DPyPmoeUdcE): Conference presentation video


## Required Preperation
Please watch the following videos before starting the lab (each is pretty short):
* [Introduction to Parallel Computing](https://youtu.be/RNVIcm8-6RE)
* [Amdahl's Law](https://youtu.be/Axx2xuB-Xuo)
* [CPU Caching](https://youtu.be/KmairurdiaY)
* [Pipelining](https://youtu.be/zPmfprtdzCE)
* [Instruction Level Parallelism](https://youtu.be/ZoDUI3gTkDI)
* [Introduction to SIMD](https://youtu.be/o_n4AKwdfiA)


### Exercise 0

Please run the following cells.  Examine the result of the example function that makes use of the *timer* wrapper. Also please use this timer wrapper (defined below) for all the exercises that follow.

#### Load python modules (Note: you may need to install some Python packages for the modules below, e.g. Numba)

In [5]:
import numba
import numpy as np


from functools import wraps
from time import time, sleep
from itertools import permutations

#### Define a function-decorator for timing the runtime of your functions

Below is some code that you can use to wrap your functions so that you can time them individually.  The function defined imeadiately below the *timer()* is an example of how to use the warpper. In the cell that follows, execute this example function. (Note, for this timer, we're are making use of something called *decorators*, but a discussion about this feature is outside the scope of this class.)


In [38]:
# defining a function decorator for timing other functions
def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):4.6f}s\n')
        return result
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

#### Run the example function

Run the example function for each of the following instances of $N = 10^5, 10^6, 10^7, 10^8$. Please examine the results, in particular, how the runtime changes with respect to $N$.


**Answer** the following questions in the markdown cell that follow the code. 
* For each factor-of-ten increase in $N$, roughly how much longer was the runtime of the function?
* Does this slowdown in the runtime make sense? Why or why not?

In [39]:
# Call the example function above for each value of N (try making one-call first, then loop)
N = [10e5, 10e6, 10e7, 10e8]
res=[]
for n in N:
    res.append(example_sum_timer_wrap(n))

Function: 'example_sum_timer_wrap({})' executed in 0.005166s

Function: 'example_sum_timer_wrap({})' executed in 0.060029s

Function: 'example_sum_timer_wrap({})' executed in 0.536034s

Function: 'example_sum_timer_wrap({})' executed in 20.114775s



#### Your answer for Exerciese 0 questions here:



**Response**: 
From N=10e5 to N=10e6, the time is ~8 times longer.
From N=10e6 to N=10e7, the time is ~17 times longer.
From N=10e7 to N=10e8, the time is ~10 times longer.

In general, the slowdown in the runtime make sense to me. With the increase of the size of array, the computational time also increase almost linearly. But I can find that the increasing rate is not strictly linear to that of the data size. 

<br><br>
### Exercise 1: Naive matrix multiplication: 

For this exercise, we will write our own matrx-matrix multiplication function.  We are going to be naive about it, so we want to loop over individual indices, instead of using slicing (in the next section, we will parallelize this code and we want to see how well Numba does at speeding up the naive code).

#### Tasks for this exercise

1. Write a function with that calculates matrix-matrix multiplication such that $C = A\cdot B$, where $A$, $B$, and $C$ are 2D numpy arrays. Make the dtype for the $C$ matrix the same as the dtype for $A$. Below is a stencil you can start with. Test your results for accuracy and performance against the np.dot() function. To time the np.dot() function, you can wrap it in another function and use the *mytimer* wrapper; however, please copy the code for testing the A and B dimensions into your function wrapper for the np.dot() function.  This keeps the runtime comparison as a more "apples-to-apples" like comparison.

```python
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zeros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
<br>

2. Define your functions in the cell below.
3. Test and compare the accuracy and runtime of your functions in the next cell.
    - Test with non-square Matrices: $A \in \mathbb{R}^{N\times K}$ and $B \in \mathbb{R}^{K \times M}$. With $N = 64$, $K = 32$, and $M = 128$.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 64, 128,$ and $256$.
    - Test the following three cases:
        - Case-1. $A$ and $B$ **both** have dtype=np.**float32** (make sure that $C$ is also of dtype=np.**float32**)
        - Case-2. $A$ has dtype=np.**float64**, but $B$ has dtype=np.**float32**
        - Case-3. $A$ and $B$ **both** have dtype=np.**float64** (make sure that $C$ is also of dtype=np.**float64**)
    - For all three case above:
        - Calculate and show the error by computing the sum of the differnece between the $C$ matrices computed from numpy.dot() and your my_naive_matmul() functions. Assume that numpy.dot() is correct
        - Calculate and show the *speedup* that the faster function has versus the slower function.
        - Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.
4. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
A = np.random.rand(N, K)
B = np.random.rand("for-you-to-figure-out")
```

<br>

#### Write function definitions for Exercise 1 below

In [40]:
# Define your functions for Exercise 1 here.

@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    
    return C

@mytimer
def my_numpy_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a numpy matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # perform numpy matrix-multiplication
    C = np.dot(A, B)
    
    # return the 2D numpy array for C
    return C

<br>

#### Write the test codes for Exercise 1 in the cell below.

In [43]:
# Test your code here for runtime and accuracy

# Test 1
N, K, M = 64, 32, 128
A = np.random.rand(N, K)
B = np.random.rand(K, M)

C1 = my_naive_matmul(A, B)
C2 = my_numpy_matmul(A, B)

# compare two results
comp = np.any(np.isclose(C1, C2))
err  = np.sum(C1 - C2) / np.sum(C2)
print(f'Test 1: C1 = C2: {comp}, relative error {err:.4f}')

Function: 'my_naive_matmul({})' executed in 0.147035s

Function: 'my_numpy_matmul({})' executed in 0.000290s

Test 1: C1 = C2: True, relative error -0.0000


In [49]:
# Test 2
for i in [64, 128, 256]:
    N, K, M = i, i, i
    A = np.random.rand(N, K)
    B = np.random.rand(K, M)

    C1 = my_naive_matmul(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test 2: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'my_naive_matmul({})' executed in 0.160706s

Function: 'my_numpy_matmul({})' executed in 0.000288s

Test 2: C1 = C2: True, relative error -0.0000


Function: 'my_naive_matmul({})' executed in 1.153297s

Function: 'my_numpy_matmul({})' executed in 0.000288s

Test 2: C1 = C2: True, relative error -0.0000


Function: 'my_naive_matmul({})' executed in 8.748025s

Function: 'my_numpy_matmul({})' executed in 0.000385s

Test 2: C1 = C2: True, relative error -0.0000




In [50]:
# Test 3
for i in range(3):
    N, K, M = 256, 256, 256
    if i == 0:
        A = np.random.rand(N, K).astype(np.float32)
        B = np.random.rand(K, M).astype(np.float32)
    elif i == 1:
        A = np.random.rand(N, K).astype(np.float64)
        B = np.random.rand(K, M).astype(np.float32)
    else:
        A = np.random.rand(N, K).astype(np.float64)
        B = np.random.rand(K, M).astype(np.float64)
        
    C1 = my_naive_matmul(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test 3-Case{i+1}: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'my_naive_matmul({})' executed in 8.785456s

Function: 'my_numpy_matmul({})' executed in 0.000572s

Test 3-Case1: C1 = C2: True, relative error 0.0000


Function: 'my_naive_matmul({})' executed in 9.322998s

Function: 'my_numpy_matmul({})' executed in 0.000561s

Test 3-Case2: C1 = C2: True, relative error 0.0000


Function: 'my_naive_matmul({})' executed in 9.263541s

Function: 'my_numpy_matmul({})' executed in 0.000396s

Test 3-Case3: C1 = C2: True, relative error 0.0000




In [92]:
# Here, I compare the speed up between my function in these three cases
print(f'Speed up for Test 3 case1  : {8.785456/8.785456:.4f}')
print(f'Speed up for Test 3 case2  : {8.785456/9.322998:.4f}')
print(f'Speed up for Test 3 case3  : {8.785456/9.263541:.4f}')

Speed up for Test 3 case1  : 1.0000
Speed up for Test 3 case2  : 0.9423
Speed up for Test 3 case3  : 0.9484


**Response**: In my tests, I found in case 1 of Test 3, i.e., when A and B are both in the type of float32, the function achieve the best speedup. I suspect the type of float32 involves less computational cost compared with that of float64.

<br><br>
### Exercise 2: Using Numba to speed up matrix multiplication: 

For this exercise, numba to speedup our matrx-matrix multiplication function below. However, there is an interesting twist to this exercise. We will write six versions of the naive function you wrote above. One function for each of the six possible permutations of three loops used to calculate the multplication (i.e. ijk, ikj, jik, etc.)

#### The tasks for this exercise:
1. Make six copies your code for the my_naive_matmul(), one copy for each of the possible permutations and define them in the cell bellow:
    - One of these function will have the same loop order as the my_naive_matmul() function. 
    - However name these functions: numba_mul_\<perm\>(), where \<perm\> should be replaced by the specific loop order of the function.
2. In the cell that follows your function definitions, test and compare the accuracy and runtime of your functions against the numpy.dot() function.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 128, 256,$ and $512$.
    - For each matrix set their dtype as: dtype=np.float64
    - Calculate and show the error between your functions and the numpy.dot() function. (Same as in Exercise 1.)
    - Calculate and show the *speedup* that the fastest function has versus the all the other functions.
    - You should notice that one of your permutation functions is faster than the others. For this case show the following:
        - Calculate and show the *speedup* that the fastest permutation function has versus the my_naive_matmul().
        - Calculate and show the *speedup* that the fastest permutation function has when $A$, $B$, and $C$ are all of dtype=np.float64 vs all are of dtype=np.float32.
3. Create your matrices using random numbers. (Same as in Exercise 1.) 
4. For each function, you need to add a function decorator for numba. Numba will *JIT* the function (**J**ust **I**n **T**ime compilation). 
    - Use the flag to keep a "cache" of the compiled code (not to be confused with CPU cache). **Note**: to get accurate timings, you will need to run your tests **twice**, because during the first run, the code is compiled and this compile time will be included in your runtime. After the first run, the compiled binary will be stored, so consecutive runs will be faster.
    - Use the flag to use *fast math*
    - Use the flag to disable the Python Global Interpretor Lock (GIL).
    - The code below should get you started, but it is incomplete.
    
```python   
    
@mytimer
@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_<perm>(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>): # !!!! LOOK HERE !!!!
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
    
6. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the questions asked in the markdown cell.


<br>

#### Write function definitions for Exercise 2 below

In [73]:
# Define your functions for Exercise 2 here.

@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_ijk(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for i in numba.prange(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C


@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_ikj(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for i in numba.prange(C.shape[0]):
        for k in range(A.shape[1]):
            for j in range(C.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C


@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_jki(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for j in range(C.shape[1]):
        for k in range(A.shape[1]):
            for i in numba.prange(C.shape[0]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C


@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_jik(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for j in range(C.shape[1]):
        for i in numba.prange(C.shape[0]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C


@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_kij(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for k in range(A.shape[1]):
        for i in numba.prange(C.shape[0]):
            for j in range(C.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C


@mytimer
@numba.njit(cache=True, fastmath=True, nogil=False)
def numba_mul_kji(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # check that A and B have appropriate dimensions for multiplication
    if A.shape[1] != B.shape[0]:
        raise Exception("Error: the A.shape[1] and B.shape[0] should be the same!")
    
    # construnct a 2D numpy array for matrix C
    C = np.zeros((A.shape[0], B.shape[1]), dtype = A.dtype)
    
    # matrix-multiplication
    for k in range(A.shape[1]):
        for j in range(C.shape[1]):
            for i in numba.prange(C.shape[0]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C

# compile these functions
C = numba_mul_ijk(A, B)
C = numba_mul_ikj(A, B)
C = numba_mul_jik(A, B)
C = numba_mul_jki(A, B)
C = numba_mul_kij(A, B)
C = numba_mul_kji(A, B)

Function: 'numba_mul_ijk({})' executed in 0.233220s

Function: 'numba_mul_ikj({})' executed in 0.119242s

Function: 'numba_mul_jik({})' executed in 0.257444s

Function: 'numba_mul_jki({})' executed in 0.658002s

Function: 'numba_mul_kij({})' executed in 0.120405s

Function: 'numba_mul_kji({})' executed in 0.675532s



<br>

#### Write the test codes for Exercise 2 in the cell below.

In [80]:
# Test your code here for runtimes, accuracy, and speedups

# Test ijk
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_ijk(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_ijk({})' executed in 0.002635s

Function: 'my_numpy_matmul({})' executed in 0.000302s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_ijk({})' executed in 0.039000s

Function: 'my_numpy_matmul({})' executed in 0.000375s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_ijk({})' executed in 0.222690s

Function: 'my_numpy_matmul({})' executed in 0.002083s

Test: C1 = C2: True, relative error 0.0000




In [81]:
# Test ikj
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_ikj(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_ikj({})' executed in 0.002444s

Function: 'my_numpy_matmul({})' executed in 0.000315s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_ikj({})' executed in 0.020652s

Function: 'my_numpy_matmul({})' executed in 0.000430s

Test: C1 = C2: True, relative error -0.0000


Function: 'numba_mul_ikj({})' executed in 0.123050s

Function: 'my_numpy_matmul({})' executed in 0.001253s

Test: C1 = C2: True, relative error -0.0000




In [82]:
# Test jik
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_jik(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_jik({})' executed in 0.002772s

Function: 'my_numpy_matmul({})' executed in 0.000315s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_jik({})' executed in 0.034688s

Function: 'my_numpy_matmul({})' executed in 0.000228s

Test: C1 = C2: True, relative error -0.0000


Function: 'numba_mul_jik({})' executed in 0.249239s

Function: 'my_numpy_matmul({})' executed in 0.002361s

Test: C1 = C2: True, relative error -0.0000




In [83]:
# Test jki
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_jki(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_jki({})' executed in 0.005193s

Function: 'my_numpy_matmul({})' executed in 0.000083s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_jki({})' executed in 0.076767s

Function: 'my_numpy_matmul({})' executed in 0.000229s

Test: C1 = C2: True, relative error -0.0000


Function: 'numba_mul_jki({})' executed in 0.662944s

Function: 'my_numpy_matmul({})' executed in 0.002023s

Test: C1 = C2: True, relative error 0.0000




In [84]:
# Test kij
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_kij(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)
    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_kij({})' executed in 0.002028s

Function: 'my_numpy_matmul({})' executed in 0.000294s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_kij({})' executed in 0.015602s

Function: 'my_numpy_matmul({})' executed in 0.000260s

Test: C1 = C2: True, relative error -0.0000


Function: 'numba_mul_kij({})' executed in 0.121527s

Function: 'my_numpy_matmul({})' executed in 0.001299s

Test: C1 = C2: True, relative error 0.0000




In [85]:
# Test kji
for i in [128, 256, 512]:
    N, K, M = i, i, i
    A = np.random.rand(N, K).astype(np.float64)
    B = np.random.rand(K, M).astype(np.float64)

    C1 = numba_mul_kji(A, B)
    C2 = my_numpy_matmul(A, B)

    # compare two results
    comp = np.any(np.isclose(C1, C2))
    err  = np.sum(C1 - C2) / np.sum(C2)

    print(f'Test: C1 = C2: {comp}, relative error {err:.4f}\n\n')

Function: 'numba_mul_kji({})' executed in 0.004875s

Function: 'my_numpy_matmul({})' executed in 0.000211s

Test: C1 = C2: True, relative error 0.0000


Function: 'numba_mul_kji({})' executed in 0.075247s

Function: 'my_numpy_matmul({})' executed in 0.000513s

Test: C1 = C2: True, relative error -0.0000


Function: 'numba_mul_kji({})' executed in 0.694287s

Function: 'my_numpy_matmul({})' executed in 0.002402s

Test: C1 = C2: True, relative error -0.0000




In [90]:
# Here, I only compare the speed up between different permutation schemes, ignoring the numpy dot 
print(f'Speed up for permutation ijk N = 128 : {0.002028/0.002635:.4f}')
print(f'Speed up for permutation ikj N = 128 : {0.002028/0.002444:.4f}')
print(f'Speed up for permutation jik N = 128 : {0.002028/0.002772:.4f}')
print(f'Speed up for permutation jki N = 128 : {0.002028/0.005193:.4f}')
print(f'Speed up for permutation kij N = 128 : {0.002028/0.002028:.4f}')
print(f'Speed up for permutation kji N = 128 : {0.002028/0.004875:.4f}')
print('\n')
print(f'Speed up for permutation ijk N = 256 : {0.015602/0.039000:.4f}')
print(f'Speed up for permutation ikj N = 256 : {0.015602/0.020652:.4f}')
print(f'Speed up for permutation jik N = 256 : {0.015602/0.034688:.4f}')
print(f'Speed up for permutation jki N = 256 : {0.015602/0.076767:.4f}')
print(f'Speed up for permutation kij N = 256 : {0.015602/0.015602:.4f}')
print(f'Speed up for permutation kji N = 256 : {0.015602/0.075247:.4f}')
print('\n')
print(f'Speed up for permutation ijk N = 512 : {0.121527/0.222690:.4f}')
print(f'Speed up for permutation ikj N = 512 : {0.121527/0.123050:.4f}')
print(f'Speed up for permutation jik N = 512 : {0.121527/0.249239:.4f}')
print(f'Speed up for permutation jki N = 512 : {0.121527/0.662944:.4f}')
print(f'Speed up for permutation kij N = 512 : {0.121527/0.121527:.4f}')
print(f'Speed up for permutation kji N = 512 : {0.121527/0.694287:.4f}')

Speed up for permutation ijk N = 128 : 0.7696
Speed up for permutation ikj N = 128 : 0.8298
Speed up for permutation jik N = 128 : 0.7316
Speed up for permutation jki N = 128 : 0.3905
Speed up for permutation kij N = 128 : 1.0000
Speed up for permutation kji N = 128 : 0.4160


Speed up for permutation ijk N = 256 : 0.4001
Speed up for permutation ikj N = 256 : 0.7555
Speed up for permutation jik N = 256 : 0.4498
Speed up for permutation jki N = 256 : 0.2032
Speed up for permutation kij N = 256 : 1.0000
Speed up for permutation kji N = 256 : 0.2073


Speed up for permutation ijk N = 512 : 0.5457
Speed up for permutation ikj N = 512 : 0.9876
Speed up for permutation jik N = 512 : 0.4876
Speed up for permutation jki N = 512 : 0.1833
Speed up for permutation kij N = 512 : 1.0000
Speed up for permutation kji N = 512 : 0.1750


**Response**: I notice that the 'kij' permutation functions is fater than the others. The following code block is for testing this function

In [94]:
N = 256
# dtype=np.float64
A = np.random.rand(N, N).astype(np.float64)
B = np.random.rand(N, N).astype(np.float64)
C1 = numba_mul_kij(A, B)
C2 = my_naive_matmul(A, B)

# dtype=np.float32
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C1 = numba_mul_kij(A, B)
C2 = my_naive_matmul(A, B)


Function: 'numba_mul_kij({})' executed in 0.019128s

Function: 'my_naive_matmul({})' executed in 8.781124s

Function: 'numba_mul_kij({})' executed in 0.266509s

Function: 'my_naive_matmul({})' executed in 8.506088s



In [95]:
print(f'For N={N}, dtype=np.float64, the speedup that the fastest permutation function has versus the my_naive_matmul() is {8.781124/0.019128:.4f}')
print(f'For N={N}, dtype=np.float32, the speedup that the fastest permutation function has versus the my_naive_matmul() is {8.506088/0.266509:.4f}')

For N=256, dtype=np.float64, the speedup that the fastest permutation function has versus the my_naive_matmul() is 459.0717
For N=256, dtype=np.float32, the speedup that the fastest permutation function has versus the my_naive_matmul() is 31.9167


<br>

### Discussion for Exercise 2
1. What do you think is causing the differences in performance between the various permutations?
1. Which function is fastest (permutations or numpy.dot)? Why do you think this function is the fastest, and could there be multiple factors involved regarding the superior performance?
1. When comparing the matrix-matrix performance between the cases were all matrices had dtype=np.float64 vs.  all matrices having dtype=np.float32, which dtype was fastest? Roughtly what was the speedup when using this dtype vs the other?
1. Does Amdahl's Law play a major factor in the performance differences? Which part of the matrix-matrix multiplication was not parallelized (serial portion)? (Hint: which matricies did we reuse for each function?) Could we parallelize this part, and if so, are there caveats?
1. Do you think all codes should be parallelized? What about matrix-matrix multiplication? (This is a subjective question, but I am looking for a brief, but rational and informed arguement).
1. For those taking the class for **4 units**: Regarding the performance differences, which Law do you think is more relevant when comparing the performance differences between each of the functions in this exercise, Amdahl's Law or Gustafson's Law? Explain your reasoning. 